# Goals in cleaning up the data.
1. Extract GPS coordinates from Location column (street address)
2. Consolidate number of Incidents (group similar e.g Theft from Motor Vehicle vs. Theft)


In [1]:
import pandas as pd
import numpy as np
import re
from pprint import pprint

In [2]:
# what am I using?

import pkg_resources
modules_used = 'pandas,numpy,geopy,folium'.split(',')
print('Versions')
for m in modules_used:
    print('{}:\t{}'.format(m, pkg_resources.get_distribution(m).version))

Versions
pandas:	0.22.0
numpy:	1.14.0
geopy:	1.17.0
folium:	0.6.0


In [3]:
# import data
df = pd.read_json('crimeLogData.json')
df.shape

(11240, 7)

In [4]:
# see columns
df.columns

Index(['Comments', 'Disposition', 'Incident', 'Location', 'Occured',
       'Reported', 'UCPD_ID'],
      dtype='object')

In [5]:
# (ignore the fact that Occurred is misspelled smh)
df.rename(columns={'Occured':'Occurred'}, inplace=True)
df.columns

Index(['Comments', 'Disposition', 'Incident', 'Location', 'Occurred',
       'Reported', 'UCPD_ID'],
      dtype='object')

In [6]:
# peek inside
df.head()

,Comments,Disposition,Incident,Location,Occurred,Reported,UCPD_ID
0,Woman reports losing a Canon Power Shot digita...,Closed,Lost Property,5810 S. University (Quad),6/28/10 2:45 PM,7/1/10 12:42 PM,W0731
1,Unknown person smashed windshield of a rental ...,Open,Criminal Damage to Vehicle,61st & Drexel,6/29/10 to 7/1/10 3:00 PM to 1:50 PM,7/1/10 2:01 PM,W0732
2,Unknown person smashed rear passenger's side w...,Open,Theft from Motor Vehicle,60th between Ingleside & Ellis,6/29/10 to 7/1/10 5:30 PM to 12:30 PM,7/1/10 2:48 PM,W0733
3,Bicycle taken from bike rack,Open,Theft,60th and Ellis,6/23/10 to 6/29/10 9:00 AM to 5:00 PM,7/1/10 3:13 PM,W0734
4,"Bottle slipped from man's hands, broke causing...",Closed,Chemical Spill,924 E. 57th St. (BSLC),7/1/10 6:15 PM,7/1/10 6:15 PM,W0735


In [7]:
# remove bad data
'''
NaN corresponds to "No Incident Reports" & missing data, and
voids to some internal error
'''
df[9010:9020]  # preview

,Comments,Disposition,Incident,Location,Occurred,Reported,UCPD_ID
9010,"Individual reports losing iPhone, possibly on ...",Closed,Lost Property,5640 S. Ellis to 56th & Cornell,3/17/17 10:00 PM to 11:00 PM,3/18/17 10:55 AM,17-00215
9011,Individual collapsed during a track meet / Tra...,Closed,Medical Call,5550 S. University (Crown Field House),3/18/17 12:45 PM,3/18/17 12:45 PM,17-00216
9012,3 unknown suspects struck victim with fists an...,CPD,Information / Robbery / Vehicle Hijacking,5558 S. Kimbark,3/18/17 4:05 PM,3/18/17 4:07 PM,17-00217
9013,Number generated in error,Void,Void,Void,Void,Void,17-00218
9014,NaN,NaN,:,NaN,NaN,NaN,NaN
9015,NaN,NaN,No Incident Reports this date 3/19/17,NaN,NaN,NaN,NaN
9016,NaN,NaN,:,NaN,NaN,NaN,NaN
9017,11 aluminum support beams taken from bleachers...,Open,Theft,821 E. 55th St. (Stagg Field),3/17/17 to 3/20/17 2:00 PM to 10:30 AM,3/20/17 11:05 AM,17-00219
9018,U.S. passport left unattended on copy machine ...,Open,Theft,1009 E. 57th St. (Hitchcock RH),2/14/17 9:50 AM,3/20/17 12:26 PM,17-00220
9019,An unauthorized individual sleeping in the bui...,Ex. Cleared,Assault,6019 S. Ingleside (Campus Housing),3/20/17 7:50 AM,3/20/17 3:20 PM,17-00221


In [8]:
# remove nan values
df = df.dropna()

# remove voids (check for error in comments column)
void_err = lambda x: re.search(r'error', x) is None
df = df[df['Comments'].apply(void_err)]
df.shape

(10075, 7)

In [9]:
# lowercase and strip first four columns

for col in df.columns[[0, 1, 2, 3]]:
    df[col] = df[col].apply(lambda x: x.lower().strip())

## Extract GPS coordinate
We use the geopy, a geocoding web service, module to interact with Google Maps API.
We have addresses that need geolocation coordinates.

In [10]:
from geopy.geocoders import GoogleV3
from geopy.exc import GeocoderTimedOut

In [11]:
# get api-key (unique to user, get a key in the link below)
# https://developers.google.com/maps/documentation/geocoding/start
with open('google-maps-api/api-key.txt', 'r') as f:
    key = f.readline()

# setup geolocator using Google Maps API
geolocator = GoogleV3(api_key=key, user_agent="uchicago_surronding_area", format_string="%s, Chicago, IL")

In [12]:
'''
Function takes an address as a string and formats to improve 
likelihood of getting gps coordinates from geocode.

Returns:
    If location found: a tuple containing the latitude & longitude
    If not: an integer code (user defined)
'''
def address2coordinates(address):
    
    # adding "avenue" helps narrow down location
    def addAvenue(address):
        avenues = 'cottage grove,drexel,ingleside,ellis,greenwood,university,woodlawn,kimbark,\
kenwood,dorchester,blackstone,harper,lake park,stony island,cornell,everett'.split(',')
        
        for ave in avenues:
            if re.search(ave, address):
                address += ' avenue'
                break
                
        return address

    
    address = re.sub(r' \(.*\)', '', address.lower())  # remove parenthetical info
    address = re.sub(r' at|and ', ' & ', address)      # fix intersection
    
    try:
        # check if address is between streets
        # e.g. (52nd St. between Greenwood & University)
        pattern = re.findall(r'(\w.*(?= between)) between ((?<=between )\w.*(?= &)) & ((?<=\& )\w.*)', address)[0]
        street_1 = '{} {}'.format(pattern[0], pattern[1])
        street_2 = '{} {}'.format(pattern[0], pattern[2])
        location_1 = geolocator.geocode(addAvenue(street_1), timeout=10)  # server timeout time is 10sec
        location_2 = geolocator.geocode(addAvenue(street_2), timeout=10)
        
        # returns midpoint coordinates between the two locations
        # e.g. (52nd st. greenwood avenue / 52nd st. university avenue)
        return ((location_1.latitude + location_2.latitude)/2, (location_1.longitude + location_2.longitude)/2)
            
    except IndexError:
        # only single address
        location = geolocator.geocode(address, timeout=10)
        if location != None:
            return (location.latitude, location.longitude)
        
    except GeocoderTimedOut:
        # server timeout value if any (check later)
        print('Server timeout!')
    
    except:
        # error value will point to unforseen error if any (check later)
        print('An error occurred!')
        return 999
    
    return 0

In [13]:
# add a progress bar for the next cell (over 10K lines)
from tqdm._tqdm_notebook import tqdm_notebook

# change pandas apply to progress_apply
# tqdm adds minute overhead (~90ns; see tqdm documentation) 
tqdm_notebook.pandas(desc="Geocoding Progress!")  

In [12]:
# create batch and save files as checkpoints
batch = np.linspace(0, df['Location'].shape[0], 6, dtype=int) # each will have 2015 entries
for i in range(1, 6):
    coordinates = df['Location'][batch[i-1]: batch[i]].progress_apply(address2coordinates)
    coordinates.to_csv(('data/coordinates/coordinates_{}.csv'.format(i)))

HBox(children=(IntProgress(value=0, description='Geocoding Progress!', max=2015), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Geocoding Progress!', max=2015), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Geocoding Progress!', max=2015), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Geocoding Progress!', max=2015), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Geocoding Progress!', max=2015), HTML(value='')))

In [14]:
# concatenate csv files and save

# use magic command to find path to files
filenames = !find -L data -name "*.csv"

coordinates = pd.concat([pd.read_csv(file, header=None) for file in filenames])
coordinates = coordinates.set_index(0)  # fix index

In [15]:
# where are errors 0 (server timeout) and 999 (other error) if any
server_timeout = np.argwhere(coordinates.values=='0')[:, 0]
error_999 = np.argwhere(coordinates.values=='999')[:, 0]

# how many issues?
print('Number of timeouts:\t{}\nNumber of 999 errors:\t{}'.format(len(server_timeout), len(error_999)))

Number of timeouts:	7
Number of 999 errors:	0


In [16]:
# redo timeouts
redo_addresses = df['Location'].values[server_timeout]
redo_addresses

'''
REMOVE:
'Various Campus Buildings' - invalid address
220 W. - lost propert Public Transit
53rd - lost cellphone
5640 S. - lost propert Public Transit

WHY:
Invalid Address or wide street range
'''

array(['various campus buildings', '220 w. garfield to 1000 e. 55th st.',
       '53rd & kimbark to 60th & ingleside',
       '900 e. 57th st. (knapp center)', '5250 s. harper ct.',
       '5640 s. ellis to 56th & cornell',
       '5525 s. cottage grove (public way)'], dtype=object)

In [17]:
# get geolocation again
redo_list = [address2coordinates(arr) for arr in redo_addresses]
redo_list

[0,
 0,
 0,
 (41.7916469, -87.6034081),
 (41.7996959, -87.5894821),
 0,
 (41.794078, -87.60576200000001)]

In [18]:
# pd.to_csv wrote data as a string
# that's why we are converting here
# easy fix using eval() will follow below
redo_list = np.array(list(map(str, redo_list)))[:, np.newaxis]

coordinates.iloc[server_timeout] = redo_list
coordinates.iloc[server_timeout]

,1
0,
107,0
3467,0
4052,0
4593,"(41.7916469, -87.6034081)"
6449,"(41.7996959, -87.5894821)"
9010,0
11063,"(41.794078, -87.60576200000001)"


In [19]:
# change string tuple coordinates to tuple with numberic coordinates
coordinates = coordinates[1].apply(eval)  

# save fix merge
coordinates.to_csv('data/coordinates/merged_coordinates.csv')

In [20]:
# add all coordinates to dataframe
df['Coordinates'] = coordinates.values
df.shape

(10075, 8)

In [21]:
#drop rows with coordinate = 0
df = df[df['Coordinates'] != 0]
df['Coordinates'].shape

(10071,)

In [22]:
# some voids weren't picked up 
# remove them
df = df[df['Disposition']!='void']
df.shape

(10032, 8)

In [23]:
# reset index since all bad data is dropped
df.reset_index(drop=True)
df.head()

,Comments,Disposition,Incident,Location,Occurred,Reported,UCPD_ID,Coordinates
0,woman reports losing a canon power shot digita...,closed,lost property,5810 s. university (quad),6/28/10 2:45 PM,7/1/10 12:42 PM,W0731,"(41.7892236, -87.5980197)"
1,unknown person smashed windshield of a rental ...,open,criminal damage to vehicle,61st & drexel,6/29/10 to 7/1/10 3:00 PM to 1:50 PM,7/1/10 2:01 PM,W0732,"(41.7840731, -87.6043863)"
2,unknown person smashed rear passenger's side w...,open,theft from motor vehicle,60th between ingleside & ellis,6/29/10 to 7/1/10 5:30 PM to 12:30 PM,7/1/10 2:48 PM,W0733,"(41.785901249999995, -87.60199659999999)"
3,bicycle taken from bike rack,open,theft,60th and ellis,6/23/10 to 6/29/10 9:00 AM to 5:00 PM,7/1/10 3:13 PM,W0734,"(41.7859081, -87.6011825)"
4,"bottle slipped from man's hands, broke causing...",closed,chemical spill,924 e. 57th st. (bslc),7/1/10 6:15 PM,7/1/10 6:15 PM,W0735,"(41.7919425, -87.60283989999999)"


## Consolidate number of Incidents
There are a lot of incidents and we're interested in a few times. Let's group those that are similar (e.g theft from motor vehicle vs theft). Though similar under the law they are different, however, the main point is here is to see if there is any apparent trend.

In [24]:
# number of unique incidents
df['Incident'].unique().shape

(505,)

In [25]:
# unique incident count and incident
incident_count = [[len(df[df['Incident'] == crime]), crime] for crime in df['Incident'].unique()]
incident_count[:10]  # preview

[[649, 'lost property'],
 [84, 'criminal damage to vehicle'],
 [238, 'theft from motor vehicle'],
 [2336, 'theft'],
 [3, 'chemical spill'],
 [57, 'possession of cannabis'],
 [166, 'battery'],
 [88, 'robbery'],
 [257, 'injured person'],
 [50, 'disturbance']]

In [26]:
# incident categories
categories = 'lost,theft,assault,burglary,mental health'.split(',')
categories

['lost', 'theft', 'assault', 'burglary', 'mental health']

In [27]:
categories_data = {}
for i in categories:
    categories_data[i] = df['Incident'].str.extract(r'({})'.format(i), expand=True).dropna().index

In [28]:
df.loc[categories_data['lost'].values]

,Comments,Disposition,Incident,Location,Occurred,Reported,UCPD_ID,Coordinates
0,woman reports losing a canon power shot digita...,closed,lost property,5810 s. university (quad),6/28/10 2:45 PM,7/1/10 12:42 PM,W0731,"(41.7892236, -87.5980197)"
5,staff member reports losing wallet containing ...,closed,lost property,"out of area - palos heights, il",7/1/10 10:00 PM,7/1/10 11:09 PM,W0736,"(41.66861859999999, -87.7841447)"
72,patient's cell phone missing after she transfe...,closed,lost property,5815 s. maryland (mitchell hospital),7/18/10 2:00 PM to 7:00 PM,7/18/10 7:19 PM,W0803,"(41.7892116, -87.60406619999999)"
111,patient reports a hp computer missing after sh...,closed,lost property,5815 s. maryland (mitchell hospital),7/24/10 11:00 AM to 7:05 PM,7/24/10 7:07 PM,W0842,"(41.7892116, -87.60406619999999)"
122,patient reports loss of eye glasses,closed,lost property,5815 s. maryland (mitchell hospital),7/23/10 12:00 PM to 6:00 PM,7/25/10 1:41PM,W0852,"(41.7892116, -87.60406619999999)"
128,staff member reports losing her office keys,closed,lost property,924 e. 57th st. (bslc),7/26/10 1:00 PM to 2:30 PM,7/26/10 4:31 PM,W0859,"(41.7919425, -87.60283989999999)"
153,woman lost her wallet while in the cafeteria,closed,lost property,5758 s. maryland (dcam),7/29/10 1:30 PM to 4:00 PM,7/29/10 5:54 PM,W0884,"(41.7899689, -87.6052594)"
173,ucpd officers observed two males attempting to...,arrest,theft of lost or mislaid property,1100 e. 57th st. (regenstein library),8/2/10 5:30 PM,8/2/10 5:46 PM,W0904,"(41.792279, -87.599954)"
182,radiation sensor lost while working at the site,closed,lost property,5800 s. stony island (doctors hospital),7/18/10 12:40 PM,8/4/10 6:45 PM,W0912,"(41.78962, -87.58657500000001)"
212,ucpd officer arrested man for possession of a ...,arrest,possession of lost or stolen property,harper between 54th & 55th,8/10/10 11:35 PM,8/10/10 11:38 PM,W0943,"(41.79659625, -87.58874639999999)"


In [68]:
#df_categories = pd.concat([df.loc[categories_data[cat].values] for cat in categories])
#for cat in categories:
#    df_categories = pd.concat(df.loc[categories_data[cat].values], axis=1)

## Visualization w/ folium

In [29]:
# plot coordinates using folium, python wrapper of Leaflet (a JavaScript library for interactive maps)
import folium
from folium.plugins import HeatMap, FastMarkerCluster

In [30]:
# set
hyde_park = [41.7943, -87.5907]
m = folium.Map(location=hyde_park, zoom_start=14, tiles='Stamen Toner')

for entry, comment in zip(df['Coordinates'].values[10026:], df["Comments"].values[10026:]):
    folium.Circle(entry, radius=5, color='crimson', fill=False, popup=folium.Popup(comment)).add_to(m)


HeatMap(df['Coordinates'].apply(list).values[10026:].tolist()).add_to(m)    
m

In [46]:
m.save('commented_heatmap.html')

In [31]:
crime_map = folium.Map(location=hyde_park, zoom_start=14, tiles='Stamen Toner')

for entry, comment in zip(df['Coordinates'].values[10026:], df["Comments"].values[10026:]):
    folium.Circle(entry, radius=5, color='crimson', fill=False, popup=folium.Popup(comment)).add_to(crime_map)


HeatMap(df['Coordinates'].apply(list).values[10026:].tolist()).add_to(crime_map)    

crime_map

In [32]:
# let's make folium easier to use
def folium_api(*args, **kwargs):
#    map_ = folium.Map(location=)
    print(args)
    print(kwargs)
    return 0


In [33]:
# initialize map
crime_map = folium.Map(location=hyde_park, zoom_start=14, min_zoom=14)

# string representation of a valid Javascript function 
# used to update default icon appearance (taken from github example)
callback = """\
function (row) {
    var icon, marker;
    icon = L.AwesomeMarkers.icon({
        icon: "map-marker", markerColor: "red"});
    marker = L.marker(new L.LatLng(row[0], row[1]));
    marker.setIcon(icon);
    return marker;
};
"""
# add_to map (e.g data and info)
FastMarkerCluster(data=(df.loc[categories_data['theft']])['Coordinates'], callback=callback).add_to(crime_map)

# display updated map
crime_map

In [37]:
#for entry in df['Coordinates'].loc[categories_data['assault'].values],:
 #   print(entry)
 #   break
m = folium.Map(location=[41.7943, -87.5907], zoom_start=14, tiles='Stamen toner', min_zoom=14)

HeatMap(df['Coordinates'].loc[categories_data['assault'].values], radius=17).add_to(m)
m

In [34]:
# get rows w/ index labels matching w/ 'assault'
#(df.loc[categories_data['assault']])['Coordinates']

In [35]:
#((df.loc[categories_data['assault']]).groupby('Incident'))

In [36]:
"""
# definition of the boundaries in the map
district_geo = r'sfpddistricts.geojson'
  
# calculating total number of incidents per district
crimedata2 = pd.DataFrame(crimedata['PdDistrict'].value_counts().astype(float))
crimedata2.to_json('crimeagg.json')
crimedata2 = crimedata2.reset_index()
crimedata2.columns = ['District', 'Number']
  
# creation of the choropleth
map1 = folium.Map(location=SF_COORDINATES, zoom_start=12)
map1.geo_json(geo_path = district_geo, 
              data_out = 'crimeagg.json', 
              data = crimedata2,
              columns = ['District', 'Number'],
              key_on = 'feature.properties.DISTRICT',
              fill_color = 'YlOrRd', 
              fill_opacity = 0.7, 
              line_opacity = 0.2,
              legend_name = 'Number of incidents per district')
               
display(map1)
"""

"\n# definition of the boundaries in the map\ndistrict_geo = r'sfpddistricts.geojson'\n  \n# calculating total number of incidents per district\ncrimedata2 = pd.DataFrame(crimedata['PdDistrict'].value_counts().astype(float))\ncrimedata2.to_json('crimeagg.json')\ncrimedata2 = crimedata2.reset_index()\ncrimedata2.columns = ['District', 'Number']\n  \n# creation of the choropleth\nmap1 = folium.Map(location=SF_COORDINATES, zoom_start=12)\nmap1.geo_json(geo_path = district_geo, \n              data_out = 'crimeagg.json', \n              data = crimedata2,\n              columns = ['District', 'Number'],\n              key_on = 'feature.properties.DISTRICT',\n              fill_color = 'YlOrRd', \n              fill_opacity = 0.7, \n              line_opacity = 0.2,\n              legend_name = 'Number of incidents per district')\n               \ndisplay(map1)\n"